In [ ]:
# configs
storage_account_name = "[storage_account_name]"
tenant_id = "[tenant_id]"
client_id = "[client_id]"
client_secret = "[client_secret]"

# spark configuration options
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net","OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [ ]:
from pyspark.sql.functions import col
from datetime import datetime

dataverse_customer_table_name = '[dataverse_customer_table_name]'
customer_spend = 'customer_spend'
silver_container_name = 'silver'
gold_customer_table_name = 'customer'
gold_container_name = 'gold'
partition_date = datetime.now().strftime("%Y-%m-%d")

customer_spend_df = spark.read.parquet(f"abfss://{silver_container_name}@{storage_account_name}.dfs.core.windows.net/{customer_spend}")
dataverse_customer_df = spark.read.parquet(f"abfss://{silver_container_name}@{storage_account_name}.dfs.core.windows.net/{dataverse_customer_table_name}")

customer_df = dataverse_customer_df.join(customer_spend_df,dataverse_customer_df.cid ==  customer_spend_df.cid,"inner").drop(customer_spend_df.cid)

customer_df.write.mode('append').parquet(f"abfss://{gold_container_name}@{storage_account_name}.dfs.core.windows.net/{gold_customer_table_name}")

In [ ]:
%sql

CREATE TABLE if not exists hive_metastore.default.customer
                   USING PARQUET
                   LOCATION 'abfss://gold@[storage_account_name].dfs.core.windows.net/customer'
